<a name='top'></a>
Javier Lopez<br>
Student ID: 000697446
# Prepare NYC Rideshare, Weather, Covid, and Vaccination Data
## Feb 2019-Mar 2022
***

In [1]:
import os
import time
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('mode.chained_assignment', None)

## Main Function
***

In [2]:
def main():
    process_tic = time.perf_counter()
    clean_tripdata()
    resample_tripdata()
    concat_dfs()
    clean_taxi_zones()
    uber = pd.read_parquet('data/clean_data/uber_taxi_zone_tripdata.parquet')
    uber.name = 'uber'
    lyft = pd.read_parquet('data/clean_data/lyft_taxi_zone_tripdata.parquet')
    lyft.name = 'lyft'
    [get_borough_tripdata(df) for df in [uber, lyft]]
    midprocess_toc = time.perf_counter()
    print('Rideshare data cleaning completed in {:.2f} minutes' .format((midprocess_toc - process_tic)/60))
    clean_weather_data()
    clean_covid_data()
    clean_vax_data()
    process_toc = time.perf_counter()
    print('All data cleaning completed in {:.2f} minutes' .format((process_toc - process_tic)/60))

## Clean Rideshare Data
***

### By Taxi Zone
***

In [3]:
def clean_df(df, df_name):
    # Set index to pickup datetime
    df.set_index('pickup_datetime', drop=True, inplace=True)
    df.index.name = None

    # Make all columns lowercase
    df.columns = [df[col].name.lower() for col in df.columns]

    # Keep only study columns
    df = df[['hvfhs_license_num', 'pulocationid']]

    # Split by provider
    uber = df[['pulocationid']][df.hvfhs_license_num == 'HV0003']
    lyft = df[['pulocationid']][df.hvfhs_license_num == 'HV0005']
    del df

    # Create dummy variables for taxi zones
    uber.pulocationid, lyft.pulocationid = [df.pulocationid.astype(str) for df in [uber, lyft]]
    uber, lyft = [pd.concat([df, pd.get_dummies(df.pulocationid)], axis=1) for df in [uber, lyft]]
    
    # Drop pulocationid column
    uber, lyft = [df.drop('pulocationid', axis=1) for df in [uber, lyft]]
    
    # Rename columns
    uber.columns = ['zone_' + col for col in uber.columns]
    lyft.columns = ['zone_' + col for col in lyft.columns]

    # Drop 'EWR' and 'Unknown' taxi zones
    drop_zones = ['zone_1', 'zone_264', 'zone_265']
    for zone in drop_zones:
        for df in [uber, lyft]:
            if zone in df.columns:
                df.drop(zone, axis=1, inplace=True)

    uber.to_parquet('data/rideshare_data/clean_rideshare_data/uber_{}.parquet' .format(df_name))
    lyft.to_parquet('data/rideshare_data/clean_rideshare_data/lyft_{}.parquet' .format(df_name))

In [4]:
# Load tripdata files
def clean_tripdata():
    load_clean_tic = time.perf_counter()
    for root, dirs, files in os.walk('./data/rideshare_data', topdown=False):
        for name in files:
            if name != '.DS_Store':
                filename = name[6:22].replace('-', '_')
                # Load file
                load_tic = time.perf_counter()
                df = pd.read_parquet(os.path.join(root, name))
                load_toc = time.perf_counter()
                print(filename, 'loading completed in {:.2f} minutes' .format((load_toc - load_tic)/60))
                # Clean file
                clean_tic = time.perf_counter()
                clean_df(df, filename)
                clean_toc = time.perf_counter()
                print(filename, 'cleaning completed in {:.2f} minutes' .format((clean_toc - clean_tic)/60))
    load_clean_toc = time.perf_counter()
    print('All file cleaning completed in {:2f} minutes' .format((load_clean_toc - load_clean_tic)/60))

In [5]:
def resample_tripdata():
    tic = time.perf_counter()
    for root, dirs, files in os.walk('./data/rideshare_data/clean_rideshare_data', topdown=False):
        for name in files:
            if name != '.DS_Store':
                filename = name[:-8]
                # Load file
                load_tic = time.perf_counter()
                df = pd.read_parquet(os.path.join(root, name))
                load_toc = time.perf_counter()
                print(filename, 'loading completed in {:.2f} minutes' .format((load_toc - load_tic)/60))
                # Resample by day
                resample_tic = time.perf_counter()
                df.index = pd.to_datetime(df.index).normalize()
                df = df.resample('D').agg(np.sum)
                df.to_parquet('data/rideshare_data/resampled_rideshare_data/{}.parquet' .format(filename))
                resample_toc = time.perf_counter()
                print(filename, 'resampling completed in {:.2f} minutes' .format((resample_toc - resample_tic)/60))
    toc = time.perf_counter()
    print('All resampling completed in {:2f} minutes' .format((toc - tic)/60))

In [6]:
# Define function to concatenate dataframes
def concat_dfs():
    tic = time.perf_counter()
    uber = pd.DataFrame()
    lyft = pd.DataFrame()
    for root, dirs, files in os.walk('./data/rideshare_data/resampled_rideshare_data', topdown=False):
        for name in files:
            if name != '.DS_Store':
                # Load file
                df = pd.read_parquet(os.path.join(root, name))
                if 'uber' in name:
                    uber = pd.concat([uber, df])
                    print(name, 'loaded into uber dataframe')
                if 'lyft' in name:
                    lyft = pd.concat([lyft, df])
                    print(name, 'loaded into lyft dataframe')
    uber.sort_index(inplace=True)
    lyft.sort_index(inplace=True)
    uber.to_parquet('data/clean_data/uber_taxi_zone_tripdata.parquet')
    lyft.to_parquet('data/clean_data/lyft_taxi_zone_tripdata.parquet')
    toc = time.perf_counter()
    print('Concatenation completed in {:.2f} minutes' .format((toc - tic)/60))

### By Borough
***

In [7]:
def clean_taxi_zones():
    # Load taxi zones
    taxi_zones = pd.read_csv('data/taxi_zones.csv')
    
    # Make columns lowercase
    taxi_zones.columns = [taxi_zones[col].name.lower() for col in taxi_zones]
    
    # Select necessary columns
    taxi_zones = taxi_zones[['locationid', 'borough']]
    
    taxi_zones.to_csv('data/clean_data/taxi_zones_clean.csv', index=False)
    print('File cleaning completed')

In [8]:
def get_borough_tripdata(df):
    # Load taxi zones
    taxi_zones = pd.read_csv('data/clean_data/taxi_zones_clean.csv')
    
    # List boroughs
    boroughs = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']
    
    # Get taxi zones for each borough
    zone_dict = {}
    for borough in boroughs:
        zone_dict[borough] = taxi_zones.locationid.values[taxi_zones.borough == borough]
    
    # Make dataframe for each borough
    borough_dfs = {}
    for borough in boroughs:
        borough_df = pd.DataFrame()
        for num in zone_dict[borough]:
            for col in df.columns:
                if col == 'zone_' + str(num):
                    borough_df[col] = df[col]
        borough_dfs[borough] = borough_df
    
    # Make dataframe of borough aggregation
    borough_tripdata = pd.DataFrame(index=df.index)
    for key, value in borough_dfs.items():
        borough_tripdata[key.lower().replace(' ', '_')] = [value.loc[value.index[i]].sum() for i in range(len(value))]
    
    # Add nyc total column
    borough_tripdata['nyc'] = [
        borough_tripdata.loc[borough_tripdata.index[i]].sum() for i in range(len(borough_tripdata))]
    
    borough_tripdata.to_parquet('data/clean_data/{}_borough_tripdata.parquet' .format(df.name))
    print('Aggregation completed for', df.name)

## Clean Weather Data
***

In [9]:
def clean_weather_data():
    # Load file
    df = pd.read_csv('data/weather.csv')

    # Set index to datetime
    df.set_index(pd.to_datetime(df.DATE), inplace=True)
    df.drop('DATE', axis=1, inplace=True)
    df.index.name = None

    # Make columns lowercase
    df.columns = [col.lower() for col in df.columns]

    # Select necessary columns
    df = df[[ 'prcp', 'snow', 'tmax', 'tmin']]
    df.columns = ['precipitation', 'snowfall', 'max_temp', 'min_temp']

    # Get average temperature
    df['temp'] = (df.max_temp + df.min_temp) / 2

    # Select necessary columns
    df = df[['temp', 'precipitation', 'snowfall']]

    # Rename columns
    df.columns = ['temp', 'prec', 'snow']
    
    # Select study dates
    df = df['2019-02-01':'2022-03-31']
    
    df.to_parquet('data/clean_data/weather_data_clean.parquet')
    print('Weather data cleaning completed')

## Clean Covid Data
***

In [10]:
def clean_covid_data():
    # Load file
    df = pd.read_csv('data/covid_boroughs_case_hosp_death.csv')
    
    # Set index to datetime
    df.set_index(pd.to_datetime(df.DATE_OF_INTEREST), inplace=True)
    df.drop('DATE_OF_INTEREST', axis=1, inplace=True)
    df.index.name = None
    
    # Make columns lowercase
    df.columns = [col.lower() for col in df.columns]
    
    # Re-encode column names
    boroughs = {'bx':'bronx','bk':'brooklyn','mn':'manhattan','qn':'queens','si':'staten_island'}
    colnames = []
    to_drop = []
    for col in df.columns:
        for key, value in boroughs.items():
            if key + '_' in col:
                col = col.replace(key + '_', value + '_')
        if '7day_avg' not in col or 'all' in col:
            to_drop.append(col)
        colnames.append(col)
    df.columns = colnames
    df.drop(to_drop, axis=1, inplace=True)
    df.columns.values[:3] = ['nyc_' + col for col in df.columns[:3]]
    df.columns = [col[:-15] for col in df.columns]
    df.columns.values[1] = df.columns.values[1][:7]+'pitalized'
    
    # Select study dates
    df = df['2019-02-01':'2022-03-31']
    
    df.to_parquet('data/clean_data/covid_data_clean.parquet')
    print('Covid data cleaning completed')

## Clean Vaccination Data
***

In [11]:
def clean_vax_data():
    # Load file
    df = pd.read_csv('data/Covid-19_Vaccinations.csv')

    # Set index to datetime
    df.set_index(pd.to_datetime(df.Date), inplace=True)
    df.drop('Date', axis=1, inplace=True)
    df.index.name = None
    df.sort_index(inplace=True)

    # Select necessary columns
    df['vaccinated'] = df['Administered_Dose1_Recip'].shift(-1)-df['Administered_Dose1_Recip']
    df = df[['vaccinated']]
    df = df['2021-04-16':'2022-03-31']

    df.to_parquet('data/clean_data/vaccination_data_clean.parquet')
    print('Vaccination data cleaning completed')

## Get Temporal Phase Dummies
***

In [12]:
root = 'data/clean_data/'
uber = pd.read_parquet(root + 'uber_borough_tripdata.parquet')
lyft = pd.read_parquet(root + 'lyft_borough_tripdata.parquet')
weather = pd.read_parquet(root + 'weather_data_clean.parquet')
covid = pd.read_parquet(root + 'covid_data_clean.parquet')
vax = pd.read_parquet(root + 'vaccination_data_clean.parquet')

In [13]:
phases = pd.DataFrame(index=uber.index, columns=['phase'])

In [14]:
phases.phase['2019-02-01':'2020-03-10'] = ['phase_0']*len(phases.phase['2019-02-01':'2020-03-10'])
phases.phase['2020-03-11':'2020-05-19'] = ['phase_1']*len(phases.phase['2020-03-11':'2020-05-19'])
phases.phase['2020-05-20':'2021-04-15'] = ['phase_2']*len(phases.phase['2020-05-20':'2021-04-15'])
phases.phase['2021-04-16':] = ['phase_3']*len(phases.phase['2021-04-16':])

In [15]:
phases = pd.get_dummies(phases.phase)

In [16]:
phases.head(3)

,phase_0,phase_1,phase_2,phase_3
2019-02-01,1,0,0,0
2019-02-02,1,0,0,0
2019-02-03,1,0,0,0


In [17]:
phases.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1155 entries, 2019-02-01 to 2022-03-31
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   phase_0  1155 non-null   uint8
 1   phase_1  1155 non-null   uint8
 2   phase_2  1155 non-null   uint8
 3   phase_3  1155 non-null   uint8
dtypes: uint8(4)
memory usage: 45.8 KB


##  Condense Data by Borough
***

In [18]:
def concat_vars(borough):
    df = pd.concat([uber[[borough]], lyft[[borough]]], axis=1)
    df.columns = ['uber', 'lyft']
    df = pd.concat([df, weather], axis=1)
    cols = [col for col in covid.columns if borough in col]
    df = pd.concat([df, covid[cols]], axis=1).fillna(0)
    df = pd.concat([df, vax], axis=1).fillna(0)
    df = pd.concat([df, phases], axis=1).fillna(0)
    return df

In [19]:
bronx, brooklyn, manhattan, queens, staten_island, nyc = [concat_vars(borough) for borough in uber.columns]

In [20]:
iterables = [
    uber.columns.values,
    ['uber','lyft','temp','prec','snow','case','hospitalized','death','vaccination',
     'phase_0', 'phase_1', 'phase_2', 'phase_3']
]
columns = pd.MultiIndex.from_product(iterables)
data = pd.DataFrame(index=uber.index, columns=columns)

In [21]:
for borough in uber.columns:
    data[borough] = globals()[borough].values

In [22]:
data.head()

bronx                                                   \
               uber     lyft  temp prec snow case hospitalized death   
2019-02-01  73444.0  13319.0  16.0  0.0  0.0  0.0          0.0   0.0   
2019-02-02  73752.0   9929.0  25.0  0.0  0.0  0.0          0.0   0.0   
2019-02-03  67072.0   9087.0  43.0  0.0  0.0  0.0          0.0   0.0   
2019-02-04  59847.0   9852.0  51.0  0.0  0.0  0.0          0.0   0.0   
2019-02-05  51170.0   9239.0  54.5  0.0  0.0  0.0          0.0   0.0   

                                ...  nyc                               \
           vaccination phase_0  ... prec snow case hospitalized death   
2019-02-01         0.0     1.0  ...  0.0  0.0  0.0          0.0   0.0   
2019-02-02         0.0     1.0  ...  0.0  0.0  0.0          0.0   0.0   
2019-02-03         0.0     1.0  ...  0.0  0.0  0.0          0.0   0.0   
2019-02-04         0.0     1.0  ...  0.0  0.0  0.0          0.0   0.0   
2019-02-05         0.0     1.0  ...  0.0  0.0  0.0          0.0   0.0   

                                                        
           vaccination phase_0 phase_1 phase_2 phase_3  
2019-02-01         0.0     1.0     0.0     0.0     0.0  
2019-02-02         0.0     1.0     0.0     0.0     0.0  
2019-02-03         0.0     1.0     0.0     0.0     0.0  
2019-02-04         0.0     1.0     0.0     0.0     0.0  
2019-02-05         0.0     1.0     0.0     0.0     0.0  

[5 rows x 78 columns]

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1155 entries, 2019-02-01 to 2022-03-31
Data columns (total 78 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   (bronx, uber)                  1155 non-null   float64
 1   (bronx, lyft)                  1155 non-null   float64
 2   (bronx, temp)                  1155 non-null   float64
 3   (bronx, prec)                  1155 non-null   float64
 4   (bronx, snow)                  1155 non-null   float64
 5   (bronx, case)                  1155 non-null   float64
 6   (bronx, hospitalized)          1155 non-null   float64
 7   (bronx, death)                 1155 non-null   float64
 8   (bronx, vaccination)           1155 non-null   float64
 9   (bronx, phase_0)               1155 non-null   float64
 10  (bronx, phase_1)               1155 non-null   float64
 11  (bronx, phase_2)               1155 non-null   float64
 12  (bronx, phase_3)              

In [24]:
data.to_parquet('data/clean_data/model_dataset.parquet')